# Clean and Analyze Employee Exit Surveys

## Introduction

dataset
- exit surveys of employees from Queensland, Australia
    - Department of Education, Training and Employment (DETE)
    - Technical and Further Education (TAFE)
    - encoded to UTF-8

project goal
- Are employes who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction?
- What about the employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction?
- What about older employees?

- combine results for both surveys to answer the quetions
- use same survey template, but one customized some of the answers
- no data dictionary available

skills:
- apply(), map()
- fillna(), dropna(), drop()
- melt()
- concat(), merge()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. The DETE and TAFE Survey Datasets

`dete_survey.csv`
* `ID` participant ID
* `SeparationType` reason why employment ended
* `Cease Date` year or month employment ended
* `DETE Start Date` year employemnt started

`tafe_survey.csv`
* `Record ID` participant ID
* `Reason for ceasing employment`
* `LengthofServiceOverall. Overall Length of Service at Institute (in years)` employment in years

In [ ]:
# # read in and preview datasets
# dete_raw = pd.read_csv('/dete_survey.csv')
# tafe_raw = pd.read_csv('/tafe_survey.csv')

dete_raw = pd.read_csv('/Users/slp22/code/dataquest projects/tafe_survey.csv')
tafe_raw = pd.read_csv('/Users/slp22/code/dataquest projects/dete_survey.csv')

### DETE

In [ ]:
dete_raw.head()

,ID,SeparationType,Cease Date,DETE Start Date,Role Start Date,Position,Classification,Region,Business Unit,Employment Status,...,Kept informed,Wellness programs,Health & Safety,Gender,Age,Aboriginal,Torres Strait,South Sea,Disability,NESB
0,1,Ill Health Retirement,08/2012,1984,2004,Public Servant,A01-A04,Central Office,Corporate Strategy and Peformance,Permanent Full-time,...,N,N,N,Male,56-60,NaN,NaN,NaN,NaN,Yes
1,2,Voluntary Early Retirement (VER),08/2012,Not Stated,Not Stated,Public Servant,AO5-AO7,Central Office,Corporate Strategy and Peformance,Permanent Full-time,...,N,N,N,Male,56-60,NaN,NaN,NaN,NaN,NaN
2,3,Voluntary Early Retirement (VER),05/2012,2011,2011,Schools Officer,NaN,Central Office,Education Queensland,Permanent Full-time,...,N,N,N,Male,61 or older,NaN,NaN,NaN,NaN,NaN
3,4,Resignation-Other reasons,05/2012,2005,2006,Teacher,Primary,Central Queensland,NaN,Permanent Full-time,...,A,N,A,Female,36-40,NaN,NaN,NaN,NaN,NaN
4,5,Age Retirement,05/2012,1970,1989,Head of Curriculum/Head of Special Education,NaN,South East,NaN,Permanent Full-time,...,N,A,M,Female,61 or older,NaN,NaN,NaN,NaN,NaN


In [ ]:
# dete_raw.info()

In [ ]:
# dete_raw.columns

In [ ]:
# dete_raw.isnull()

In [ ]:
dete_raw['SeparationType'].value_counts()

,count
SeparationType,
Age Retirement,285
Resignation-Other reasons,150
Resignation-Other employer,91
Resignation-Move overseas/interstate,70
Voluntary Early Retirement (VER),67
Ill Health Retirement,61
Other,49
Contract Expired,34
Termination,15


In [ ]:
dete_raw['Position'].value_counts()

,count
Position,
Teacher,324
Teacher Aide,137
Public Servant,126
Cleaner,97
Head of Curriculum/Head of Special Education,38
Schools Officer,24
School Administrative Staff,16
Guidance Officer,12
Technical Officer,11


In [ ]:
# dete_raw['Classification'].value_counts()

**`dete_raw`**
- RangeIndex: 822 entries, 0 to 821
- Data columns (total 56 columns)
- Dytpe: ID=int, others=object, bool
- Non-Null: Business Unit, Aboriginal, Torres Strait, South Sea, Disability, NESB

### TAFE

In [ ]:
tafe_raw.head()

,Record ID,Institute,WorkArea,CESSATION YEAR,Reason for ceasing employment,Contributing Factors. Career Move - Public Sector,Contributing Factors. Career Move - Private Sector,Contributing Factors. Career Move - Self-employment,Contributing Factors. Ill Health,Contributing Factors. Maternity/Family,...,Workplace. Topic:Does your workplace promote a work culture free from all forms of unlawful discrimination?,Workplace. Topic:Does your workplace promote and practice the principles of employment equity?,Workplace. Topic:Does your workplace value the diversity of its employees?,Workplace. Topic:Would you recommend the Institute as an employer to others?,Gender. What is your Gender?,CurrentAge. Current Age,Employment Type. Employment Type,Classification. Classification,LengthofServiceOverall. Overall Length of Service at Institute (in years),LengthofServiceCurrent. Length of Service at current workplace (in years)
0,6.341330e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2010.0,Contract Expired,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Female,26 30,Temporary Full-time,Administration (AO),1-2,1-2
1,6.341337e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010.0,Retirement,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
2,6.341388e+17,Mount Isa Institute of TAFE,Delivery (teaching),2010.0,Retirement,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,6.341399e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010.0,Resignation,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,6.341466e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010.0,Resignation,-,Career Move - Private Sector,-,-,-,...,Yes,Yes,Yes,Yes,Male,41 45,Permanent Full-time,Teacher (including LVT),3-4,3-4


In [ ]:
# tafe_raw.info()

In [ ]:
# tafe_raw.columns

In [ ]:
# tafe_raw.isnull()

In [ ]:
tafe_raw['Reason for ceasing employment'].value_counts()

,count
Reason for ceasing employment,
Resignation,340
Contract Expired,127
Retrenchment/ Redundancy,104
Retirement,82
Transfer,25
Termination,23


In [ ]:
tafe_raw['Employment Type. Employment Type'].value_counts()

,count
Employment Type. Employment Type,
Permanent Full-time,237
Temporary Full-time,177
Contract/casual,71
Permanent Part-time,59
Temporary Part-time,52


In [ ]:
# tafe_raw['Classification. Classification'].value_counts()

**`tafe_raw`**
- Record ID in scientific notation
- Columns names are long, descriptive, repetitive
- RangeIndex: 702 entries, 0 to 701
- Data columns (total 72 columns)
- Dtype: ID=int, others=object, cessation year=float
- Non-Null: range 400-500 of 700 rows

## 2. Identify Missing Values and Drop Unnecessary Columns

In [ ]:
# dete_raw = pd.read_csv('/dete_survey.csv', na_values="Not Stated")

dete_raw = pd.read_csv('/Users/slp22/code/dataquest projects/tafe_survey.csv')


In [ ]:
#dete_raw drop columns [28:49] axis=1
dete = dete_raw.drop(dete_raw.columns[28:49],axis=1)
dete.head()

,Record ID,Institute,WorkArea,CESSATION YEAR,Reason for ceasing employment,Contributing Factors. Career Move - Public Sector,Contributing Factors. Career Move - Private Sector,Contributing Factors. Career Move - Self-employment,Contributing Factors. Ill Health,Contributing Factors. Maternity/Family,...,Workplace. Topic:Does your workplace promote a work culture free from all forms of unlawful discrimination?,Workplace. Topic:Does your workplace promote and practice the principles of employment equity?,Workplace. Topic:Does your workplace value the diversity of its employees?,Workplace. Topic:Would you recommend the Institute as an employer to others?,Gender. What is your Gender?,CurrentAge. Current Age,Employment Type. Employment Type,Classification. Classification,LengthofServiceOverall. Overall Length of Service at Institute (in years),LengthofServiceCurrent. Length of Service at current workplace (in years)
0,6.341330e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2010.0,Contract Expired,NaN,NaN,NaN,NaN,NaN,...,Yes,Yes,Yes,Yes,Female,26 30,Temporary Full-time,Administration (AO),1-2,1-2
1,6.341337e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010.0,Retirement,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
2,6.341388e+17,Mount Isa Institute of TAFE,Delivery (teaching),2010.0,Retirement,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,6.341399e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010.0,Resignation,-,-,-,-,-,...,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,6.341466e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010.0,Resignation,-,Career Move - Private Sector,-,-,-,...,Yes,Yes,Yes,Yes,Male,41 45,Permanent Full-time,Teacher (including LVT),3-4,3-4


In [ ]:
#tafe drop columns [17:66] axis=1
tafe = tafe_raw.drop(tafe_raw.columns[17:66], axis=1)
tafe.head()

,ID,SeparationType,Cease Date,DETE Start Date,Role Start Date,Position,Classification,Region,Business Unit,Employment Status,Career move to public sector,Career move to private sector,Interpersonal conflicts,Job dissatisfaction,Dissatisfaction with the department,Physical work environment,Lack of recognition
0,1,Ill Health Retirement,08/2012,1984,2004,Public Servant,A01-A04,Central Office,Corporate Strategy and Peformance,Permanent Full-time,True,False,False,True,False,False,True
1,2,Voluntary Early Retirement (VER),08/2012,Not Stated,Not Stated,Public Servant,AO5-AO7,Central Office,Corporate Strategy and Peformance,Permanent Full-time,False,False,False,False,False,False,False
2,3,Voluntary Early Retirement (VER),05/2012,2011,2011,Schools Officer,NaN,Central Office,Education Queensland,Permanent Full-time,False,False,False,False,False,False,False
3,4,Resignation-Other reasons,05/2012,2005,2006,Teacher,Primary,Central Queensland,NaN,Permanent Full-time,False,True,False,False,False,False,False
4,5,Age Retirement,05/2012,1970,1989,Head of Curriculum/Head of Special Education,NaN,South East,NaN,Permanent Full-time,False,False,False,False,False,False,False


### Dropped columns from `tafe` [28:39] and `dete` [17:66] that are not relevant to this analysis. And will make the data easier to work with.  

## 3. Clean Column Names

In [ ]:
dete_col = dete.columns

In [ ]:
tafe_col = tafe.columns

### 🧹 functions to clean up text

In [ ]:
# function to make each column name lowercase
def lower(cols):
  lower_cols = []
  for c in cols:
    lower_cols.append(c.lower())
  return lower_cols

In [ ]:
lower(dete_col)

['id',
 'separationtype',
 'cease date',
 'dete start date',
 'role start date',
 'position',
 'classification',
 'region',
 'business unit',
 'employment status',
 'career move to public sector',
 'career move to private sector',
 'interpersonal conflicts',
 'job dissatisfaction',
 'dissatisfaction with the department',
 'physical work environment',
 'lack of recognition',
 'lack of job security',
 'work location',
 'employment conditions',
 'maternity/family',
 'relocation',
 'study/travel',
 'ill health',
 'traumatic incident',
 'work life balance',
 'workload',
 'none of the above',
 'gender',
 'age',
 'aboriginal',
 'torres strait',
 'south sea',
 'disability',
 'nesb']

In [ ]:
# function to remove trailing whitespace from end of strings
def spaceless(cols):
  spaceless_cols = []
  for c in cols:
    spaceless_cols.append(c.rstrip())
  return spaceless_cols

In [ ]:
spaceless(dete_col)

['ID',
 'SeparationType',
 'Cease Date',
 'DETE Start Date',
 'Role Start Date',
 'Position',
 'Classification',
 'Region',
 'Business Unit',
 'Employment Status',
 'Career move to public sector',
 'Career move to private sector',
 'Interpersonal conflicts',
 'Job dissatisfaction',
 'Dissatisfaction with the department',
 'Physical work environment',
 'Lack of recognition',
 'Lack of job security',
 'Work location',
 'Employment conditions',
 'Maternity/family',
 'Relocation',
 'Study/Travel',
 'Ill Health',
 'Traumatic incident',
 'Work life balance',
 'Workload',
 'None of the above',
 'Gender',
 'Age',
 'Aboriginal',
 'Torres Strait',
 'South Sea',
 'Disability',
 'NESB']

In [ ]:
# function to replace space with underscore
def replace_punctuation(cols):
  underscore_cols = []
  for c in cols:
    new_c = c.replace(" ", "_").replace(".", "").replace("-", "")
    underscore_cols.append(new_c)
  return underscore_cols

In [ ]:
replace_punctuation(dete_col)

['ID',
 'SeparationType',
 'Cease_Date',
 'DETE_Start_Date',
 'Role_Start_Date',
 'Position',
 'Classification',
 'Region',
 'Business_Unit',
 'Employment_Status',
 'Career_move_to_public_sector',
 'Career_move_to_private_sector',
 'Interpersonal_conflicts',
 'Job_dissatisfaction',
 'Dissatisfaction_with_the_department',
 'Physical_work_environment',
 'Lack_of_recognition',
 'Lack_of_job_security',
 'Work_location',
 'Employment_conditions',
 'Maternity/family',
 'Relocation',
 'Study/Travel',
 'Ill_Health',
 'Traumatic_incident',
 'Work_life_balance',
 'Workload',
 'None_of_the_above',
 'Gender',
 'Age',
 'Aboriginal',
 'Torres_Strait',
 'South_Sea',
 'Disability',
 'NESB']

### 🚫 apply clean up functions #1 (nested functions)

In [ ]:
# from types import new_class
def clean_up(col):
    new_cols = []

  # function to make each column name lowercase

#   # function to remove trailing whitespace from end of strings
#     def spaceless(new_cols):
#         for c in new_cols:
#             new_cols.append(c.rstrip())
#         return new_cols

#   # function to replace space with underscore
#     def replace_punctuation(new_cols):
#         for c in new_cols:
#             new_c = c.replace(" ", "_").replace(".", "").replace("-", "")
#             new_cols.append(new_c)
#         return new_cols
    return lower(new_cols


In [ ]:
# higer order function lesson

def generate_age_checker(min_age):
    def check_age(age):
        return age > min_age
    return check_age

check_min_18 = generate_age_checker(18)
check_min_21 = generate_age_checker(21)

print(check_min_18(20))
print(check_min_21(20))
    

True
False


### ✅ apply clean up functions #2 (sequential)



In [ ]:
# appy lower, spaceless, and replace_punctuation functions for tafe_col
lower_tafe = lower(tafe_col)
spaceless_tafe = spaceless(lower_tafe)
clean_tafe_cols = replace_punctuation(spaceless_tafe)
# print(clean_tafe_cols)

In [ ]:
# appy lower, spaceless, and replace_punctuation functions for dete_col
lower_dete = lower(dete_col)
spaceless_dete = spaceless(lower_dete)
clean_dete_cols = replace_punctuation(spaceless_dete)
# print(clean_dete_cols)

### ✅ apply clean up functions #3 (nest func)

In [ ]:
# replace_punctuation(spaceless(lower(dete_col)))

In [ ]:
# replace_punctuation(spaceless(lower(tafe_col)))

### ✅ apply clean up functions #4 (call func)

In [ ]:
# best practice
def clean_up(col):
    lowercased = lower(col)
    without_spaces = spaceless(lowercased)
    without_punctuation = replace_punctuation(without_spaces)
    return without_punctuation

In [ ]:
clean_up(dete_col)

In [ ]:
clean_up(tafe_col)

['id',
 'separationtype',
 'cease_date',
 'dete_start_date',
 'role_start_date',
 'position',
 'classification',
 'region',
 'business_unit',
 'employment_status',
 'career_move_to_public_sector',
 'career_move_to_private_sector',
 'interpersonal_conflicts',
 'job_dissatisfaction',
 'dissatisfaction_with_the_department',
 'physical_work_environment',
 'lack_of_recognition']

# 🎬 [3 of 11 · Clean Column Names](https://app.dataquest.io/c/60/m/348/guided-project%3A-clean-and-analyze-employee-exit-surveys/3/clean-column-names?path=2&slug=data-scientist&version=2.5)

`tafe_survey_updated`

'Record ID': 'id'
'CESSATION YEAR': 'cease_date'
'Reason for ceasing employment': 'separationtype'
'Gender. What is your Gender?': 'gender'
'CurrentAge. Current Age': 'age'
'Employment Type. Employment Type': 'employment_status'
'Classification. Classification': 'position'
'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'institute_service'
'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'role_service'